In [33]:
import sys
sys.path.append("..")  # add parent directory to system path

import pandas as pd
import json
from model.model_loader import get_model
from transformers import BartTokenizer

from model.brain_translator_model import BrainTranslator
from handler.inference import infer
from handler.generate_masks import generate_masks_from_embeddings
from handler.handler import process_uploaded_file
from transformers import BartForConditionalGeneration

tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")


In [13]:
max_file_number=50
for i in range(1, max_file_number + 1):
    # Step 3: Dynamically build the file paths based on the loop index
    file = f"../datasets/saved_data/input_embeddings_{i}.json"
    print(input_embeddings_path)
    # Step 4: Load the data from these files

./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/saved_data/input_embeddings_50.json
./datasets/sa

In [18]:
import torch

def load_embeddings_from_file(filepath: str) -> torch.Tensor:
    """
    Load embeddings from a given JSON file.

    Parameters:
    - filepath (str): The path to the JSON file containing embeddings.

    Returns:
    - torch.Tensor: A tensor containing the loaded embeddings.
    """
    with open(filepath, 'r') as file:
        embeddings_data = json.load(file)
    return torch.tensor(embeddings_data)

In [19]:
def generate_masks_from_embeddings(embeddings: torch.Tensor) -> (torch.Tensor, torch.Tensor):
    """
    Generate attention masks and their inverse for a given embeddings tensor.

    Parameters:
    - embeddings (torch.Tensor): The embeddings tensor.

    Returns:
    - tuple: A tuple containing the attention mask and its inverse.
    """
    # Assuming non-zero embeddings represent valid tokens and zeros represent padding
    attn_mask = (embeddings.sum(dim=-1) != 0).float()
    attn_mask_invert = 1.0 - attn_mask
    return attn_mask, attn_mask_invert

In [35]:
results = []
running_loss = 0.0
pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Iterate over data.
sample_count = 0

# Create a placeholder token
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
placeholder_token = tokenizer("<s>", return_tensors="pt")
target_tokens_list = []
target_string_list = []
pred_tokens_list = []
pred_string_list = []
max_file_number=50
for i in range(1, max_file_number + 1):
    # Step 3: Dynamically build the file paths based on the loop index
    file = f"../datasets/saved_data/input_embeddings_{i}.json"# Process the uploaded EEG data file
    input_embeddings_data =load_embeddings_from_file(file)

    # Generate the necessary masks
    attn_mask, attn_mask_invert = generate_masks_from_embeddings(
    input_embeddings_data
    )

    # Acquire the model and generate text
    model = BrainTranslator(pretrained_bart)
    # Step 5: Process the data with the model
    with torch.no_grad():
        try:
            outputs = model(input_embeddings_tensor, input_masks_tensor, input_mask_invert_tensor, placeholder_token["input_ids"])
            # Extract the generated token IDs from the model's outputs
            logits=outputs.logits
            probs = logits[0].softmax(dim = 1)
            values, predictions = probs.topk(1)
            predictions = torch.squeeze(predictions)
            predicted_string = tokenizer.decode(predictions).split('</s></s>')[0].replace('<s>','')
            predictions = predictions.tolist()
            truncated_prediction = []
            for t in predictions:
                if t != tokenizer.eos_token_id:
                    truncated_prediction.append(t)
                else:
                    break
            pred_tokens = tokenizer.convert_ids_to_tokens(truncated_prediction, skip_special_tokens = True)
            # print('predicted tokens:',pred_tokens)
            pred_tokens_list.append(pred_tokens)
            pred_string_list.append(predicted_string)
            print('predicted string:',predicted_string)
            # results.append(generated_text)
        except Exception as e:
            results.append(str(e))

predicted string: TheThepper
predicted string: ent
predicted string: ItItIt
predicted string: 
predicted string: 
predicted string: 
predicted string: ItItIt
predicted string: i
predicted string: 
predicted string: ItItIt
predicted string: TheThe.
predicted string: ItItIt
predicted string: ItIt it
predicted string: PP Doctor
predicted string: 
predicted string:  back
predicted string: is
predicted string: ItItIt
predicted string: III
predicted string:  Deal
predicted string:  back
predicted string: TheTheThe
predicted string: al
predicted string: .
predicted string: ItItIt
predicted string: is
predicted string: 
predicted string: is
predicted string: 
predicted string:  Beach
predicted string: ott
predicted string:  i
predicted string:  men
predicted string: ine
predicted string: ItItIt
predicted string:  Johnson
predicted string: 
predicted string: ,
predicted string: 
predicted string: 
predicted string:  model
predicted string: 
predicted string: rep
predicted string: III
predicted 

In [36]:
results

[]